In [1]:
import numpy as np
import pandas as pd
import utm
import math
import pickle
import os
import sys
sys.path.append('./src')
import time
import datetime
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import copy

import display_osm
import plot
import tools
import uni_hmm
import celltower as ctcrawler
from data_cleaning import noise, load, counter

In [2]:
load = reload(load)
file_2g = './data/jiading/data_2g.csv'
gongcan_2g = './data/jiading/2g_gongcan.csv'
final_gongcan_2g = './data/jiading/final_2g_gongcan.csv'

In [3]:
db, db_gps, towers = load.load_2g(file_2g, gongcan_2g, merge_tower=False, neighbor=True, with_rssi=False, radio_angle=True, context=False)
print 'len(db):', len(db)

Unique Cell Tower:  81
Duplicated cell towers: [[66, 67, 68], [35, 36], [53, 54], [69, 70, 71], [40, 41], [56, 57], [61, 62, 63]]
Totally duplicate: 86
len(db): 73


In [8]:
r_towers = dict()
for t, info in towers.iteritems():
    if info[-2]!=-1:
        r_towers[info[0]] = info[1:]
len(r_towers)

26

In [28]:
distribute = dict()
summary = []
for tr_id, traj in db.iteritems():
    for idx, point in enumerate(traj):
        if not r_towers.has_key(point[2]):
            continue
        tower_id = point[2]
        x1, y1 = point[0], point[1]
        lat, lng, azimuth, _ = r_towers[point[2]]
        x0, y0, _, _ = utm.from_latlon(lat, lng)
        v1 = (x1 - x0, y1 - y0)
        v0 = (0, 1)
        cos_theta = (y1 - y0) / math.sqrt((x1-x0)**2 + (y1-y0)**2)
        theta = math.acos(cos_theta) / math.pi * 180
        if v1[0] < 0:
            theta = 360 - theta
        if not distribute.has_key(tower_id):
            distribute[tower_id] = []
        angle_dist = abs(theta - azimuth)
        summary.append((tower_id, min(angle_dist, 360 - angle_dist)))
        distribute[tower_id].append(min(angle_dist, 360 - angle_dist))

In [14]:
for tower_id, distri in distribute.iteritems():
#     if tower_id!=34:
#         continue
    print 'TowerID=%d, (%d, %d)' % (tower_id, min(distri), max(distri))

TowerID=34, (23, 23)
TowerID=35, (15, 18)
TowerID=36, (15, 33)
TowerID=39, (0, 55)
TowerID=40, (118, 179)
TowerID=41, (0, 77)
TowerID=52, (27, 27)
TowerID=53, (34, 49)
TowerID=54, (67, 87)
TowerID=55, (111, 179)
TowerID=56, (38, 43)
TowerID=57, (0, 29)
TowerID=59, (61, 79)
TowerID=61, (0, 122)
TowerID=62, (0, 116)
TowerID=63, (0, 179)
TowerID=65, (63, 160)
TowerID=66, (0, 56)
TowerID=67, (0, 93)
TowerID=68, (15, 63)
TowerID=69, (0, 110)
TowerID=70, (0, 71)
TowerID=71, (0, 173)


In [ ]:
plt.bar(range(len(distribute), height=distribute.values()))


In [29]:
data = pd.DataFrame(summary, columns=['TowerID', 'Angle'])

In [30]:
data.head()

,TowerID,Angle
0,62,88.815487
1,62,47.198898
2,63,171.336734
3,63,172.425409
4,63,172.890593


In [32]:
import seaborn as sns
sns.set(style="ticks")
data = pd.DataFrame(summary, columns=['TowerID', 'Angle'])
sns.boxplot(x="TowerID", y="Angle", data=data, palette="PRGn")
sns.despine(offset=10, trim=True)

In [34]:
plt.savefig('./data/jiading/result/azimuth.png')